In [47]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D

model=Sequential()

model.add(Conv2D(256, (3, 3),input_shape=[50,50,1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))

model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [48]:
model.load_weights('../res/Models/modelAgeGender.h5')

In [49]:
import cv2, pickle
import dlib
import numpy as np

In [50]:
cam = cv2.VideoCapture('rtsp://192.168.1.6/live/ch00_1')#rtsp://192.168.1.6/live/ch00_1 'https://192.168.1.5:8080/video' '../res/test_data/videos/vid.mp4'

dict_file = open("../res/Data/cat_dict.pkl", "rb")
category_dict = pickle.load(dict_file)

face_detector = dlib.get_frontal_face_detector()
margin = 20

In [51]:
while(True):
    ret, feed = cam.read()
    feed = cv2.resize(feed, (852, 480))
    
    faces = face_detector(feed)
    
    try:
        for face in faces:

            x1 = (face.left()) - margin
            y1 = (face.top()) - margin
            x2 = (face.right()) + margin
            y2 = (face.bottom()) + margin

            test_img = feed[y1:y2, x1:x2]
            s_test = test_img

            test_img = cv2.resize(test_img, (50,50))
            test_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
            test_img = test_img/255
            test_img = test_img.reshape(1, 50, 50, 1)

            results = model.predict(test_img)
            label = np.argmax(results, axis=1)[0]
            acc = int(np.max(results, axis=1)[0]*100)

            print(results,label,category_dict[label],acc)
            
            cv2.imwrite(f"../res/Live_Face/{category_dict[label]}({acc}%).jpg", s_test)

            cv2.rectangle(feed, (x1, y1), (x2, y2),(0, 255, 0), 2)

            cv2.putText(feed, f"{category_dict[label]}({str(acc)}%)", (x1, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

    except:
        pass
    
    cv2.imshow('LIVE_FEED',feed)
    k=cv2.waitKey(1)
    if(k==27):
        break
cv2.destroyAllWindows()

[[2.0540681e-06 9.9999762e-01 3.7321042e-15 3.0422845e-07]] 1 27-40 99
[[3.7551559e-11 1.0000000e+00 1.5325229e-13 1.6097536e-09]] 1 27-40 100
[[6.2540479e-02 9.3514305e-01 2.3164344e-03 7.3296853e-08]] 1 27-40 93
[[2.6937040e-02 3.2597480e-10 8.5692391e-19 9.7306293e-01]] 3 40+ 97
[[9.9931598e-01 6.8050233e-04 5.4793006e-12 3.5440580e-06]] 0 13-26 99
[[1.8215485e-10 9.9977750e-01 7.3608957e-12 2.2254765e-04]] 1 27-40 99
[[8.0276754e-07 9.3331355e-01 5.7595062e-12 6.6685706e-02]] 1 27-40 93
[[8.7227750e-01 1.2752888e-01 4.8721834e-08 1.9359076e-04]] 0 13-26 87
[[1.8831751e-06 2.2698018e-04 9.9977118e-01 4.1297676e-10]] 2 4-12 99
[[7.4968010e-04 9.9925035e-01 3.4780397e-12 1.0405372e-08]] 1 27-40 99
[[1.33345777e-04 9.23469126e-01 7.62927085e-02 1.04858525e-04]] 1 27-40 92
[[9.1023260e-09 9.9579787e-01 7.3905173e-17 4.2022103e-03]] 1 27-40 99
[[3.1177252e-15 9.9998820e-01 2.2304384e-11 1.1827349e-05]] 1 27-40 99
[[5.5736723e-07 9.9832493e-01 5.6838693e-05 1.6176712e-03]] 1 27-40 99
[[3.